In [1]:
import copy
import pandas as pd
import numpy as np
import sys
sys.path.append("../")

from cfmining.algorithms import MAPOFCEM
from cfmining.predictors import GeneralClassifier_Shap, GeneralClassifier, TreeClassifier
from cfmining.action_set import ActionSet
from cfmining.baselines import Bruteforce, MAPOCAM, Nice, Dice
from cfmining.criteria import *

from experiments_helper import get_data_model, run_experiments, format_df_table, summarize_results, get_action_set, summarize_results_multi


%load_ext autoreload
%autoreload 2

SEED = 0

In [2]:
max_changes = 3
objective = "abs_diff"

## 1 objective

### MAPOFCEM

In [3]:
for dataset_name in ["german", "taiwan", "adult"]:
    dataset, X_train, Y_train, model, outlier_detection, individuals = get_data_model(dataset_name, "LGBMClassifier_simple")
    individuals = individuals.sample(n = 50, random_state=SEED)
    outlier_detection.contamination = dataset.outlier_contamination
    action_set = get_action_set(dataset, X_train, default_step_size=0.05)

    model_wrap = GeneralClassifier_Shap(
        model,
        outlier_detection,
        X_train,
        shap_explainer="tree",
    )

    method = MAPOFCEM(
        action_set = action_set,
        classifier = model_wrap,
        compare = objective,
        max_changes = max_changes,
        outlier_contamination = dataset.outlier_contamination,
        estimate_outlier=True,
        time_limit=np.inf,
    )

    run_experiments(
        method,
        individuals=individuals,
        model=model_wrap,
        output_file=f"../results/lgbm/{dataset}/mapofcem_tree_{objective}.csv"
    );


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:10<00:00,  4.91it/s]


### MAPOCAM

In [ ]:
for dataset_name in ["german", "taiwan", "adult"]:
    dataset, X_train, Y_train, model, outlier_detection, individuals = get_data_model(dataset_name, "LGBMClassifier_simple")
    individuals = individuals.sample(n = 50, random_state=SEED)

    outlier_detection.contamination = dataset.outlier_contamination
    action_set = get_action_set(dataset, X_train, default_step_size=0.05)
    for feat in action_set:
        feat.flip_direction = 1
        feat.update_grid()

    model_wrap = TreeClassifier(
        classifier = model,
        X = X_train,
        y = Y_train,
        use_predict_max = True,
        clf_type = "lightgbm",
    )

    method = MAPOCAM(
        action_set = action_set,
        model = model_wrap,
        criteria = objective,
        max_changes = max_changes,
    )

    run_experiments(
        method,
        individuals=individuals,
        model=model_wrap,
        output_file=f"../results/lgbm/{dataset_name}/mapocam_tree_{objective}.csv"
    );


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [01:13<00:00,  1.47s/it]
PermutationExplainer explainer: 101it [00:11,  1.14it/s]                                                                                                                                                 
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:33<00:00,  1.51it/s]


### NICE

In [ ]:
for dataset_name in ["german", "taiwan", "adult"]:
    dataset, X_train, Y_train, model, outlier_detection, individuals = get_data_model(dataset_name, "LGBMClassifier_simple")
    individuals = individuals.sample(n = 50, random_state=SEED)
    outlier_detection.contamination = dataset.outlier_contamination

    model_wrap = GeneralClassifier(
        model,
        outlier_detection,
        X_train,
    )

    method = Nice(
        X_train,
        Y_train,
        model = model,
        cat_features = dataset.categoric_features,
    )

    run_experiments(
        method,
        individuals = individuals,
        model = model_wrap,
        output_file=f"../results/lgbm/{dataset_name}/nice.csv"
    );

### DICE

In [ ]:
for dataset_name in ["german", "taiwan", "adult"]:
    dataset, X_train, Y_train, model, outlier_detection, individuals = get_data_model(dataset_name, "LGBMClassifier_simple")
    individuals = individuals.sample(n = 50, random_state=SEED)
    outlier_detection.contamination = dataset.outlier_contamination

    model_wrap = GeneralClassifier(
        model,
        outlier_detection,
        X_train,
    )


    method = Dice(
        X_train,
        Y_train,
        model,
        n_cfs = 1,
        mutable_features = dataset.mutable_features,
        continuous_features = dataset.continuous_features,
    )

    run_experiments(
        method,
        individuals = individuals,
        model = model_wrap,
        output_file=f"../results/lgbm/{dataset_name}/dice.csv"
    )


### Results

In [39]:
method_list = ["mapofcem_tree_abs_diff", "mapocam_tree_abs_diff", "mapocam_abs_diff", "dice", "nice"]

In [40]:
dataset = "german"
results = []
for method in method_list:
    results_cur = pd.read_csv(f"../results/lgbm/{dataset}/{method}.csv")
    results_cur = summarize_results(results_cur, dataset)
    results_cur["method"] = method
    results.append(results_cur)
results = pd.concat(results)
format_df_table(results, "method", results.columns.tolist()[:-1])

,method,percentile_costs,lp_costs,max_dist_costs,abs_diff_costs,n_changes,outlier,diversity,n_solutions,time
0,dice,0.444 (+-0.282) | 0.886,3.315 (+-1.572) | 5.653,2.868 (+-1.323) | 4.588,0.16 (+-0.083) | 0.292,1.74 (+-0.443) | 2.0,0.12 (+-0.328) | 1.0,0.0 (+-0.0) | 0.0,1.0 (+-0.0) | 1.0,0.158 (+-0.006) | 0.169
1,mapocam_abs_diff,0.187 (+-0.178) | 0.496,0.572 (+-0.476) | 1.404,0.56 (+-0.466) | 1.404,0.022 (+-0.019) | 0.057,1.12 (+-0.328) | 2.0,0.04 (+-0.198) | 0.0,0.0 (+-0.0) | 0.0,1.0 (+-0.0) | 1.0,1.462 (+-1.944) | 5.601
2,mapocam_tree_abs_diff,0.187 (+-0.178) | 0.496,0.572 (+-0.476) | 1.404,0.56 (+-0.466) | 1.404,0.022 (+-0.019) | 0.057,1.12 (+-0.328) | 2.0,0.04 (+-0.198) | 0.0,0.0 (+-0.0) | 0.0,1.0 (+-0.0) | 1.0,1.459 (+-1.738) | 4.746
3,mapofcem_tree_abs_diff,0.193 (+-0.182) | 0.496,0.619 (+-0.537) | 1.66,0.603 (+-0.521) | 1.652,0.025 (+-0.022) | 0.067,1.14 (+-0.405) | 2.0,0.04 (+-0.198) | 0.0,0.0 (+-0.0) | 0.0,1.0 (+-0.0) | 1.0,0.943 (+-1.776) | 2.674
4,nice,0.328 (+-0.217) | 0.776,1.165 (+-1.084) | 3.307,1.156 (+-1.076) | 3.307,0.045 (+-0.043) | 0.124,1.16 (+-0.422) | 2.0,0.04 (+-0.198) | 0.0,0.0 (+-0.0) | 0.0,1.0 (+-0.0) | 1.0,0.007 (+-0.002) | 0.01


In [41]:
dataset = "taiwan"
results = []
for method in method_list:
    results_cur = pd.read_csv(f"../results/lgbm/{dataset}/{method}.csv")
    results_cur = summarize_results(results_cur, dataset)
    results_cur["method"] = method
    results.append(results_cur)
results = pd.concat(results)
format_df_table(results, "method", results.columns.tolist()[:-1])

,method,percentile_costs,lp_costs,max_dist_costs,abs_diff_costs,n_changes,outlier,diversity,n_solutions,time
0,dice,0.599 (+-0.241) | 0.926,16.959 (+-14.615) | 46.606,16.756 (+-14.646) | 46.499,1.66 (+-1.391) | 4.44,1.66 (+-0.479) | 2.0,0.8 (+-0.404) | 1.0,0.0 (+-0.0) | 0.0,1.0 (+-0.0) | 1.0,0.219 (+-0.008) | 0.234
1,mapocam_abs_diff,0.154 (+-0.139) | 0.427,0.175 (+-0.142) | 0.437,0.159 (+-0.127) | 0.383,0.02 (+-0.018) | 0.053,1.66 (+-0.745) | 3.0,0.1 (+-0.303) | 1.0,0.0 (+-0.0) | 0.0,1.0 (+-0.0) | 1.0,0.798 (+-1.751) | 2.789
2,mapocam_tree_abs_diff,0.154 (+-0.139) | 0.427,0.175 (+-0.142) | 0.437,0.159 (+-0.127) | 0.383,0.02 (+-0.018) | 0.053,1.66 (+-0.745) | 3.0,0.1 (+-0.303) | 1.0,0.0 (+-0.0) | 0.0,1.0 (+-0.0) | 1.0,4.11 (+-5.498) | 13.667
3,mapofcem_tree_abs_diff,0.172 (+-0.166) | 0.441,0.201 (+-0.209) | 0.583,0.182 (+-0.172) | 0.53,0.023 (+-0.029) | 0.06,1.66 (+-0.798) | 3.0,0.1 (+-0.303) | 1.0,0.0 (+-0.0) | 0.0,1.0 (+-0.0) | 1.0,0.691 (+-1.76) | 1.284
4,nice,0.188 (+-0.143) | 0.493,0.335 (+-0.333) | 1.074,0.314 (+-0.315) | 1.074,0.037 (+-0.039) | 0.121,2.02 (+-0.958) | 4.0,0.06 (+-0.24) | 0.55,0.0 (+-0.0) | 0.0,1.0 (+-0.0) | 1.0,0.016 (+-0.004) | 0.024


In [43]:
dataset = "adult"
results = []
for method in method_list:
    results_cur = pd.read_csv(f"../results/lgbm/{dataset}/{method}.csv")
    results_cur = summarize_results(results_cur, dataset)
    results_cur["method"] = method
    results.append(results_cur)
results = pd.concat(results)
format_df_table(results, "method", results.columns.tolist()[:-1])

,method,percentile_costs,lp_costs,max_dist_costs,abs_diff_costs,n_changes,outlier,diversity,n_solutions,time
0,dice,0.767 (+-0.237) | 0.891,8.058 (+-3.599) | 13.791,7.743 (+-3.468) | 13.477,1.138 (+-0.551) | 2.05,1.44 (+-0.501) | 2.0,0.9 (+-0.303) | 1.0,0.0 (+-0.0) | 0.0,1.0 (+-0.0) | 1.0,0.146 (+-0.006) | 0.155
1,mapocam_abs_diff,0.707 (+-0.274) | 0.864,0.85 (+-0.331) | 1.199,0.832 (+-0.305) | 1.199,0.112 (+-0.052) | 0.162,1.14 (+-0.351) | 2.0,0.14 (+-0.351) | 1.0,0.0 (+-0.0) | 0.0,1.0 (+-0.0) | 1.0,0.396 (+-0.532) | 0.794
2,mapocam_tree_abs_diff,0.707 (+-0.274) | 0.864,0.85 (+-0.331) | 1.199,0.832 (+-0.305) | 1.199,0.112 (+-0.052) | 0.162,1.14 (+-0.351) | 2.0,0.14 (+-0.351) | 1.0,0.0 (+-0.0) | 0.0,1.0 (+-0.0) | 1.0,0.69 (+-0.387) | 1.099
3,mapofcem_tree_abs_diff,0.71 (+-0.261) | 0.864,1.031 (+-0.805) | 2.271,0.965 (+-0.646) | 2.083,0.153 (+-0.165) | 0.441,1.341 (+-0.68) | 2.85,0.0 (+-0.0) | 0.0,0.0 (+-0.0) | 0.0,0.88 (+-0.328) | 1.0,0.329 (+-0.659) | 1.489
4,nice,0.612 (+-0.344) | 0.874,0.846 (+-0.498) | 1.826,0.834 (+-0.499) | 1.826,0.11 (+-0.065) | 0.23,1.16 (+-0.422) | 2.0,0.18 (+-0.388) | 1.0,0.0 (+-0.0) | 0.0,1.0 (+-0.0) | 1.0,0.014 (+-0.002) | 0.017


## Multiple Objectives

### MAPOFCEM

In [3]:
max_changes = 3

In [4]:
for dataset_name in [
    "german",
    "taiwan", 
    "adult"
    ]:
    dataset, X_train, Y_train, model, outlier_detection, individuals = get_data_model(dataset_name, "LGBMClassifier_simple")
    individuals = individuals.sample(n = 50, random_state=SEED)
    outlier_detection.contamination = dataset.outlier_contamination
    action_set = get_action_set(dataset, X_train, default_step_size=0.05)

    model_wrap = GeneralClassifier_Shap(
        model,
        outlier_detection,
        X_train,
        shap_explainer="tree",
    )

    #setting multiple criteria
    range_calc = RangeCalculator(action_set)
    def compare_call(pivot):
        criteria_list = [
            MaxDistCriterion(
                pivot,
                range_calc,
            ),
            NumberChangesCriterion(pivot),
            AbsDiffCriterion(pivot, range_calc),
        ]
        return MultiCriterion(criteria_list, pivot)

    method = MAPOFCEM(
        action_set = action_set,
        classifier = model_wrap,
        compare = compare_call,
        max_changes = max_changes,
        outlier_contamination= dataset.outlier_contamination,
        estimate_outlier=True,
        time_limit=np.inf,
    )

    run_experiments(
        method,
        individuals=individuals,
        model=model_wrap,
        output_file=f"../results/lgbm/{dataset}/mapofcem_tree_multi.csv"
    );


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [05:07<00:00,  6.14s/it]


### MAPOCAM

In [ ]:
for dataset_name in [
    "german", 
    "taiwan", 
    "adult"
    ]:
    dataset, X_train, Y_train, model, outlier_detection, individuals = get_data_model(dataset_name, "LGBMClassifier_simple")
    individuals = individuals.sample(n = 50, random_state=SEED)
    outlier_detection.contamination = dataset.outlier_contamination
    action_set = get_action_set(dataset, X_train, default_step_size=0.05)
    for feat in action_set:
        feat.flip_direction = 1
        feat.update_grid()

    model_wrap = GeneralClassifier(
        model,
        outlier_detection,
        X_train,
    )

    #setting multiple criteria
    range_calc = RangeCalculator(action_set)

    def compare_call(pivot):
        criteria_list = [
            MaxDistCriterion(
                pivot,
                range_calc,
            ),
            NumberChangesCriterion(pivot),
            AbsDiffCriterion(pivot, range_calc),
        ]
        return MultiCriterion(criteria_list, pivot)
    
    method = MAPOCAM(
        action_set = action_set,
        model = model_wrap,
        criteria = compare_call,
        max_changes = max_changes,
    )

    run_experiments(
        method,
        individuals=individuals,
        model=model_wrap,
        output_file=f"../results/lgbm/{dataset_name}/mapocam_multi.csv"
    );


In [52]:
for dataset_name in [
    "german", 
    "taiwan", 
    "adult"
    ]:
    dataset, X_train, Y_train, model, outlier_detection, individuals = get_data_model(dataset_name, "LGBMClassifier_simple")
    individuals = individuals.sample(n = 50, random_state=SEED)
    outlier_detection.contamination = dataset.outlier_contamination
    action_set = get_action_set(dataset, X_train, default_step_size=0.05)
    for feat in action_set:
        feat.flip_direction = 1
        feat.update_grid()

    model_wrap = TreeClassifier(
        classifier = model,
        X = X_train,
        y = Y_train,
        use_predict_max = True,
        clf_type = "lightgbm",
    )

    #setting multiple criteria
    range_calc = RangeCalculator(action_set)

    def compare_call(pivot):
        criteria_list = [
            MaxDistCriterion(
                pivot,
                range_calc,
            ),
            NumberChangesCriterion(pivot),
            AbsDiffCriterion(pivot, range_calc),
        ]
        return MultiCriterion(criteria_list, pivot)
    
    method = MAPOCAM(
        action_set = action_set,
        model = model_wrap,
        criteria = compare_call,
        max_changes = max_changes,
    )

    run_experiments(
        method,
        individuals=individuals,
        model=model_wrap,
        output_file=f"../results/lgbm/{dataset_name}/mapocam_tree_multi.csv"
    );


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [01:51<00:00,  2.23s/it]
PermutationExplainer explainer: 101it [00:12,  1.66it/s]                                                                                                                                                 
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:45<00:00,  1.09it/s]


### DICE

In [ ]:
for dataset_name in ["german", "taiwan", "adult"]:
    dataset, X_train, Y_train, model, outlier_detection, individuals = get_data_model(dataset_name, "LGBMClassifier_simple")
    individuals = individuals.sample(n = 50, random_state=SEED)
    outlier_detection.contamination = dataset.outlier_contamination

    # get number of solutions of mapofcem
    #results_cur = pd.read_csv(f"../results/lgbm/{dataset_name}/mapofcem_tree_multi.csv")
    #results_cur = summarize_results(results_cur, dataset_name)
    #n_cfs = max(int(np.median(results_cur.n_solutions)), 3)
    n_cfs = 4


    model_wrap = GeneralClassifier(
        model,
        outlier_detection,
        X_train,
    )


    method = Dice(
        X_train,
        Y_train,
        model,
        n_cfs = n_cfs,
        mutable_features = dataset.mutable_features,
        continuous_features = dataset.continuous_features,
    )

    run_experiments(
        method,
        individuals = individuals,
        model = model_wrap,
        output_file=f"../results/lgbm/{dataset_name}/dice_multi.csv"
    )


In [27]:
method_list = ["mapofcem_tree_multi", "mapocam_multi", "mapocam_tree_multi", "dice_multi"]

In [28]:
dataset = "german"
results = []
for method in method_list:
    results_cur = pd.read_csv(f"../results/lgbm/{dataset}/{method}.csv")
    results_cur = summarize_results(results_cur, dataset)
    results_cur["method"] = method
    results.append(results_cur)
results = pd.concat(results)
format_df_table(results, "method", results.columns.tolist()[:-1])

,method,percentile_costs,lp_costs,max_dist_costs,abs_diff_costs,n_changes,outlier,diversity,n_solutions,time
0,dice_multi,0.453 (+-0.128) | 0.663,3.539 (+-0.745) | 4.733,3.076 (+-0.601) | 4.033,0.171 (+-0.041) | 0.233,1.76 (+-0.21) | 2.0,0.1 (+-0.222) | 0.4,0.818 (+-0.171) | 0.998,5.0 (+-0.0) | 5.0,0.177 (+-0.009) | 0.189
1,mapocam_multi,0.191 (+-0.175) | 0.542,0.593 (+-0.489) | 1.476,0.575 (+-0.462) | 1.353,0.024 (+-0.021) | 0.064,1.14 (+-0.268) | 1.5,0.04 (+-0.198) | 0.0,0.24 (+-0.431) | 1.0,1.26 (+-0.487) | 2.0,3.979 (+-6.003) | 17.077
2,mapocam_tree_multi,0.191 (+-0.175) | 0.542,0.593 (+-0.489) | 1.476,0.575 (+-0.462) | 1.353,0.024 (+-0.021) | 0.064,1.14 (+-0.268) | 1.5,0.04 (+-0.198) | 0.0,0.24 (+-0.431) | 1.0,1.26 (+-0.487) | 2.0,2.944 (+-4.483) | 11.118
3,mapofcem_tree_multi,0.193 (+-0.17) | 0.5,0.64 (+-0.552) | 1.564,0.613 (+-0.504) | 1.446,0.026 (+-0.025) | 0.072,1.178 (+-0.339) | 2.0,0.036 (+-0.179) | 0.0,0.259 (+-0.441) | 1.0,1.38 (+-0.805) | 2.55,1.482 (+-2.924) | 5.534


In [29]:
dataset = "taiwan"
results = []
for method in method_list:
    results_cur = pd.read_csv(f"../results/lgbm/{dataset}/{method}.csv")
    results_cur = summarize_results(results_cur, dataset)
    results_cur["method"] = method
    results.append(results_cur)
results = pd.concat(results)
format_df_table(results, "method", results.columns.tolist()[:-1])

,method,percentile_costs,lp_costs,max_dist_costs,abs_diff_costs,n_changes,outlier,diversity,n_solutions,time
0,dice_multi,0.624 (+-0.184) | 0.916,16.792 (+-7.176) | 28.182,16.359 (+-6.848) | 26.495,1.677 (+-0.747) | 3.133,1.655 (+-0.242) | 2.0,0.835 (+-0.172) | 1.0,0.978 (+-0.119) | 1.0,4.0 (+-0.0) | 4.0,0.233 (+-0.008) | 0.248
1,mapocam_multi,0.217 (+-0.16) | 0.496,0.386 (+-0.421) | 1.065,0.373 (+-0.411) | 1.012,0.038 (+-0.041) | 0.103,1.443 (+-0.449) | 2.0,0.127 (+-0.298) | 1.0,0.58 (+-0.498) | 1.0,1.86 (+-0.881) | 3.0,2.08 (+-4.795) | 9.489
2,mapocam_tree_multi,0.217 (+-0.16) | 0.496,0.386 (+-0.421) | 1.065,0.373 (+-0.411) | 1.012,0.038 (+-0.041) | 0.103,1.443 (+-0.449) | 2.0,0.127 (+-0.298) | 1.0,0.58 (+-0.498) | 1.0,1.86 (+-0.881) | 3.0,13.56 (+-26.109) | 63.59
3,mapofcem_tree_multi,0.205 (+-0.153) | 0.485,0.456 (+-0.549) | 1.617,0.44 (+-0.53) | 1.594,0.045 (+-0.055) | 0.155,1.423 (+-0.446) | 2.0,0.08 (+-0.234) | 0.5,0.539 (+-0.503) | 1.0,1.74 (+-0.803) | 3.0,1.973 (+-7.136) | 4.062


In [30]:
dataset = "adult"
results = []
for method in method_list:
    results_cur = pd.read_csv(f"../results/lgbm/{dataset}/{method}.csv")
    results_cur = summarize_results(results_cur, dataset)
    results_cur["method"] = method
    results.append(results_cur)
results = pd.concat(results)
format_df_table(results, "method", results.columns.tolist()[:-1])

,method,percentile_costs,lp_costs,max_dist_costs,abs_diff_costs,n_changes,outlier,diversity,n_solutions,time
0,dice_multi,0.78 (+-0.206) | 0.891,8.003 (+-1.989) | 11.303,7.646 (+-1.964) | 10.96,1.141 (+-0.29) | 1.586,1.52 (+-0.295) | 2.0,0.9 (+-0.193) | 1.0,1.0 (+-0.0) | 1.0,3.0 (+-0.0) | 3.0,0.153 (+-0.005) | 0.162
1,mapocam_multi,0.706 (+-0.277) | 0.864,0.867 (+-0.332) | 1.256,0.84 (+-0.283) | 1.235,0.116 (+-0.056) | 0.181,1.14 (+-0.268) | 1.5,0.14 (+-0.336) | 1.0,0.22 (+-0.418) | 1.0,1.24 (+-0.476) | 2.0,1.101 (+-1.042) | 2.613
2,mapocam_tree_multi,0.706 (+-0.277) | 0.864,0.867 (+-0.332) | 1.256,0.84 (+-0.283) | 1.235,0.116 (+-0.056) | 0.181,1.14 (+-0.268) | 1.5,0.14 (+-0.336) | 1.0,0.22 (+-0.418) | 1.0,1.24 (+-0.476) | 2.0,1.759 (+-1.046) | 3.544
3,mapofcem_tree_multi,0.71 (+-0.265) | 0.864,1.069 (+-0.849) | 3.101,1.004 (+-0.694) | 2.34,0.157 (+-0.17) | 0.531,1.295 (+-0.564) | 2.85,0.008 (+-0.05) | 0.0,0.159 (+-0.37) | 1.0,1.04 (+-0.57) | 2.0,1.447 (+-2.252) | 6.07
